# Books

### Leemos los ficheros

In [173]:
def regex(df):
    from pyspark.sql.functions import regexp_extract, col
    from pyspark.sql.types import StringType, BooleanType, IntegerType
    df_prima = df.select(col("Id").cast(IntegerType()), 
                      "Name", 
                      "Authors", 
                      "ISBN", 
                      col("pagesNumber").cast(IntegerType()),
                      "Language", 
                      "Publisher", 
                      col("PublishDay").cast(IntegerType()), 
                      col("PublishMonth").cast(IntegerType()), 
                      col("PublishYear").cast(IntegerType()), 
                      "Rating",
                      regexp_extract("RatingDist1", '1:(\d*)', 1).alias("RatingDist1").cast(IntegerType()), 
                      regexp_extract("RatingDist2", '2:(\d*)', 1).alias("RatingDist2").cast(IntegerType()), 
                      regexp_extract("RatingDist3", '3:(\d*)', 1).alias("RatingDist3").cast(IntegerType()), 
                      regexp_extract("RatingDist4", '4:(\d*)', 1).alias("RatingDist4").cast(IntegerType()), 
                      regexp_extract("RatingDist5", '5:(\d*)', 1).alias("RatingDist5").cast(IntegerType()),
                      regexp_extract("RatingDistTotal", 'total:(\d*)', 1).alias("RatingDistTotal").cast(IntegerType()),
                      col("CountsOfReview").cast(IntegerType())
                     )
    return df_prima

def cambio_col(df,col1, col2, nombre1, nombre2):
    df = df.withColumn("la1", col(col2))
    df = df.withColumn("la2", col(col1))
    df = df.drop(col1)
    df = df.drop(col2)
    df = df.withColumn(nombre1, col("la1"))
    df = df.withColumn(nombre2, col("la2"))
    df = df.drop("la1")
    df = df.drop("la2")
    return df
    


def dicc_df(archivos1, archivos2, ruta):
    dataframes = dict()
    for i in range(len(archivos1)):
        if i == 0:
            ruta_i = ruta+'\\'+'book' + str(archivos1[i]) + '-'+str(archivos2[i])+'k'+'.csv'
        else:
            ruta_i = ruta+'\\'+'book' + str(archivos1[i])+'k' + '-'+str(archivos2[i])+'k'+'.csv'
        #print(ruta_i)
        name = 'archivo'+'_'+str(i+1)
        arch_i = spark.read.csv(ruta_i, 
                    header=True, 
                    sep=',', 
                    quote="\"", 
                    escape="\"", 
                    ignoreTrailingWhiteSpace = True,
                    multiLine = True)
        #print(arch_i.count())
        arch_i_col = cambio_col(arch_i, "PublishDay", "PublishMonth", "PublishDay", "PublishMonth")
        arch_i_pr = regex(arch_i_col)
        #print(arch_i_pr.show(3))
        dataframes[name] = arch_i_pr
    return dataframes
    
        


def union_df(dataframes, ar):
    i=1
    for nombre, dataframe in dataframes.items():
        #print(nombre)
        if nombre != 'archivo_1':
            i+=1
            union = ar.union(dataframe)
            ar = union
    return i, ar

def nombre_ficheros():
    flag = False
    primero = [1]
    a = list(range(100, 2100, 100))
    primero.extend(a)
    primero.append(3000)
    primero.append(4000)
    segundo = list(range(100,2000, 100))
    for i in list(range(2000, 6000, 1000)):
        segundo.append(i)
    if len(primero) == len(segundo):
        flag = True
    return primero, segundo, flag

def nulos(df):
    from pyspark.sql.functions import col
    for columna in df.columns:
        print(columna)
        print(df.select(columna).where((col(columna) == '') | (col(columna).isNull())).count())
        print('\n')

def save_csv(df, ruta):
    df.write.format("dataframe.csv").save(ruta, header='true')

In [174]:
pr, se, flag = nombre_ficheros()
if flag:
    ruta = 'C:\\Users\\nora.hafidi\\Desktop\\Big Data\\UserRating\\archive'
    dfs = dicc_df(pr, se, ruta)
    ar = dfs['archivo_1']
    num, df = union_df(dfs, ar)
    print("Se han unido " + str(int(num)) + " datafames")
    print("Hay un total de "+str(df.count())+" filas")
    
else: print('Hay error el los nombres de los ficheros')

Se han unido 23 datafames
Hay un total de 1850310 filas


In [177]:
df.show(3)

+---+--------------------+------------+----------+-----------+--------+---------------+----------+------------+-----------+------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+
| Id|                Name|     Authors|      ISBN|pagesNumber|Language|      Publisher|PublishDay|PublishMonth|PublishYear|Rating|RatingDist1|RatingDist2|RatingDist3|RatingDist4|RatingDist5|RatingDistTotal|CountsOfReview|
+---+--------------------+------------+----------+-----------+--------+---------------+----------+------------+-----------+------+-----------+-----------+-----------+-----------+-----------+---------------+--------------+
|  1|Harry Potter and ...|J.K. Rowling|      null|        652|     eng|Scholastic Inc.|        16|           9|       2006|  4.57|       9896|      25317|     159960|     556485|    1546466|        2298124|         28062|
|  2|Harry Potter and ...|J.K. Rowling|0439358078|        870|     eng|Scholastic Inc.|         1|           9| 

### Contamos los nulos

In [51]:
from pyspark.sql.functions import *
nulos(df)

Id
0


Name
0


Authors
0


ISBN
5923


pagesNumber
0


Language
1598488


Publisher
17824


PublishDay
0


PublishMonth
0


PublishYear
0


Rating
0


RatingDist1
0


RatingDist2
0


RatingDist3
0


RatingDist4
0


RatingDist5
1


RatingDistTotal
1


CountsOfReview
0




In [176]:
df.schema

StructType(List(StructField(Id,IntegerType,true),StructField(Name,StringType,true),StructField(Authors,StringType,true),StructField(ISBN,StringType,true),StructField(pagesNumber,IntegerType,true),StructField(Language,StringType,true),StructField(Publisher,StringType,true),StructField(PublishDay,IntegerType,true),StructField(PublishMonth,IntegerType,true),StructField(PublishYear,IntegerType,true),StructField(Rating,StringType,true),StructField(RatingDist1,IntegerType,true),StructField(RatingDist2,IntegerType,true),StructField(RatingDist3,IntegerType,true),StructField(RatingDist4,IntegerType,true),StructField(RatingDist5,IntegerType,true),StructField(RatingDistTotal,IntegerType,true),StructField(CountsOfReview,IntegerType,true)))

In [67]:
df.columns

['Id',
 'Name',
 'Authors',
 'ISBN',
 'pagesNumber',
 'Language',
 'Publisher',
 'PublishDay',
 'PublishMonth',
 'PublishYear',
 'Rating',
 'RatingDist1',
 'RatingDist2',
 'RatingDist3',
 'RatingDist4',
 'RatingDist5',
 'RatingDistTotal',
 'CountsOfReview']

### Consultas

1. Rating promedio de todos los libros
2. Rating promedio de los libros por autor
3. Rating promedio de los libros por Publisher
4. Número promedio de páginas de todos los libros
5. Número promedio de páginas de todos los libros por autor
6. Número promedio de páginas de todos los libros por Publisher
7. Número promedio de libros publicados por autor
8. Ordenar los libros de mayor a menor (Top 15) por número de ratings dados por usuarios (excluir aquellos valores sin rating)
9. Obtener Top 5 de ratings más frecuentes otorgados por usuarios

**1.**

In [139]:
import pyspark.sql.functions as F
(df
    .select(round(F.avg("RatingDistTotal"), 2).alias("media"))
    .show()
)

+-------+
|  media|
+-------+
|4079.37|
+-------+



**2.**

In [202]:
import pyspark.sql.functions as F
(df
    .groupBy("Authors")
    .agg(round(F.avg("RatingDistTotal"), 2).alias("RankingMedio"))
    .orderBy("RankingMedio", ascending=False)
    .show())

+------------------+------------+
|           Authors|RankingMedio|
+------------------+------------+
|        Harper Lee|  4379386.33|
|      J.K. Rowling|  2980838.63|
|           Frank a|   2696635.0|
|   Stephenie Meyer|  2221830.85|
|  Kathryn Stockett|   2176064.0|
|        Anne Frank|  2116704.94|
|   Khaled Hosseini|  1879581.96|
|     Arthur Golden|  1674126.95|
|         Dan Brown|  1569989.69|
|     Stieg Larsson|  1479435.57|
|     J.D. Salinger|  1294132.54|
|      Alice Sebold|  1280483.88|
|   Cassandra Clare|  1267328.63|
|Audrey Niffenegger|   1247628.1|
|   Stephen Chbosky|   1228365.0|
|     George Orwell|   962633.51|
|      Markus Zusak|   959495.37|
|      Emily Brontë|   894090.67|
|            Homere|    853888.0|
|        Sara Gruen|   803369.64|
+------------------+------------+
only showing top 20 rows



**3.**

In [141]:
import pyspark.sql.functions as F
(df
    .groupBy("Publisher")
    .agg(round(F.avg("RatingDistTotal"),2).alias("RankingMedio"))
    .orderBy("RankingMedio", ascending=False)
    .show())

+--------------------+------------+
|           Publisher|RankingMedio|
+--------------------+------------+
|         Bloomsburry|   6788211.0|
|De Harmonie Amste...|   6765897.0|
|Raincoast Book Di...|   6742740.0|
|   Εκδόσεις Ψυχογιός|   4919788.0|
|Mañjula Pabliśiṅg...|   4604856.0|
|              Росмэн|   4553845.5|
|            Petersen|   4347616.0|
|Arthur A. Levine ...|   3886252.0|
|       Media Rodzina|  3755142.29|
|Chuoukouron Shins...|   3680023.0|
|      Mediasat Group|   3637798.0|
|              静山社|  3617272.88|
|               Tiden|   3472393.0|
|          Intrínseca|   3403592.0|
|Bloomsbury Childrens|   3300082.5|
|      人民文学出版社|  3194597.83|
| New America Library|   3147152.0|
|Houghton Brace Jo...|   3011308.0|
|Secker & Warburg ...|   2987256.0|
|      Archeion Press|  2969239.67|
+--------------------+------------+
only showing top 20 rows



**4.**

In [184]:
(df
    .select(round(F.avg("pagesNumber"), 2).alias("MediaPaginas"))
    .show())

+------------+
|MediaPaginas|
+------------+
|      276.55|
+------------+



**5.**

In [183]:
import pyspark.sql.functions as F
(df
    .groupBy("Authors")
    .agg(round(F.avg("pagesNumber"), 2).alias("MediaPaginas"))
    .orderBy("MediaPaginas", ascending=False)
    .show())

+--------------------+------------+
|             Authors|MediaPaginas|
+--------------------+------------+
|       Sandy Redburn|   1807321.6|
|         A.B. Murphy|   751507.33|
|        John B. Hare|    500000.0|
|Logos Research Sy...|    100000.0|
|Progressive Manag...|    35428.44|
|     Timothy McVeigh|     33133.0|
|   Robert H. Wozniak|     22100.0|
|Veterans Affairs ...|     16153.0|
|World Spaceflight...|    13942.33|
|         Keith Crook|      9999.0|
|             K. Muse|      9998.0|
|       Patrick Burns|      9998.0|
|         John Elstad|      9998.0|
|     Douglas    Dunn|      9998.0|
|        John Philcox|      9998.0|
|            Joe Mayo|      9998.0|
|  Albert J. Saganich|      9998.0|
|      Alice Rischert|      9998.0|
|Marian C. Merrill...|      9998.0|
|      Derbish Braber|      9998.0|
+--------------------+------------+
only showing top 20 rows



In [182]:
import pyspark.sql.functions as F
(df
    .groupBy("Authors")
    .agg(F.sum("pagesNumber").alias("MediaPaginas"))
    .orderBy("MediaPaginas", ascending=False)
    .show())

+--------------------+------------+
|             Authors|MediaPaginas|
+--------------------+------------+
|       Sandy Redburn|     9036608|
|           Anonymous|     2314273|
|         A.B. Murphy|     2254522|
|             Unknown|      741425|
|Progressive Manag...|      566855|
|        John B. Hare|      500000|
| William Shakespeare|      420507|
|Fodor's Travel Pu...|      293590|
|        Stephen King|      290827|
|     U.S. Government|      250964|
|        Nora Roberts|      236668|
|     Charles Dickens|      224916|
|Microsoft Corpora...|      206295|
|     Agatha Christie|      201075|
|Cram101 Textbook ...|      200778|
|          Mark Twain|      186553|
|        Isaac Asimov|      181400|
|National Research...|      181318|
|             Various|      176426|
|     Francine Pascal|      160393|
+--------------------+------------+
only showing top 20 rows



**6.**

In [181]:
import pyspark.sql.functions as F
(df
    .groupBy("Publisher")
    .agg(round(F.avg("pagesNumber"), 2).alias("MediaPaginas"))
    .orderBy("MediaPaginas", ascending=False)
    .show())

+--------------------+------------+
|           Publisher|MediaPaginas|
+--------------------+------------+
|Crafty Secrets Pu...|   1807321.6|
|    Sacred-texts.com|    500000.0|
|Department of Rus...|   322128.57|
|Logos Research Sy...|    100000.0|
|Encyclopedia Brit...|     32642.0|
|Progressive Manag...|    19106.36|
|Still Waters Revi...|    10080.14|
|P. Shalom Publica...|      8539.0|
|Hendrickson Publi...|      6448.0|
|            IEEE/EMB|      6000.0|
|Research Applicat...|      5856.0|
|   Hendrickson Publ.|      5808.0|
|          Childcraft|      5760.0|
|Jnanada Prakashan...|      5404.0|
|   Páginas de espuma|      5000.0|
|  Sigma Aldrich Corp|      4866.0|
| World-Mysteries.com|      4710.0|
|E B S C O Industr...|      4119.0|
|Apple Pie Publish...|      4000.0|
|     Edition Synapse|      3682.0|
+--------------------+------------+
only showing top 20 rows



**7.**

In [179]:
(df
     .groupBy("Authors")
     .count()
     .orderBy("count", ascending=False)
     .show()
)

+--------------------+-----+
|             Authors|count|
+--------------------+-----+
|           Anonymous| 2894|
|             Unknown| 2029|
| William Shakespeare| 1373|
|     Francine Pascal|  930|
|     Agatha Christie|  885|
|National Research...|  884|
|Cram101 Textbook ...|  876|
|Fodor's Travel Pu...|  858|
|        Harold Bloom|  773|
|             Various|  739|
|        Isaac Asimov|  698|
|        Nora Roberts|  647|
|       Carolyn Keene|  647|
|        Stephen King|  624|
|          R.L. Stine|  620|
| Walt Disney Company|  593|
|Hal Leonard Publi...|  588|
|          NOT A BOOK|  562|
|          Mark Twain|  526|
|       Ann M. Martin|  514|
+--------------------+-----+
only showing top 20 rows



**8.**

In [178]:
(df
     .select("Name", "Authors", "RatingDistTotal")
     .orderBy("RatingDistTotal", ascending=False)
     .show(15)
)

+-----------------------------------+------------+---------------+
|                               Name|     Authors|RatingDistTotal|
+-----------------------------------+------------+---------------+
|               Ο Χάρι Πότερ και ...|J.K. Rowling|        7094687|
|               Harry Potter og D...|J.K. Rowling|        6995593|
|               Harry Potter e a ...|J.K. Rowling|        6993152|
|               Harry Potter and ...|J.K. Rowling|        6970971|
|               Harry Potter och ...|J.K. Rowling|        6905223|
|               Harry Potter och ...|J.K. Rowling|        6871695|
|               Harry Potter e a ...|J.K. Rowling|        6865910|
|               Harry Potter and ...|J.K. Rowling|        6839208|
|               Harry Potter i Ka...|J.K. Rowling|        6833966|
|               Harri Potter maen...|J.K. Rowling|        6824640|
|ハリー・ポッターと賢者の石 (ハリ...|J.K. Rowling|        6815280|
|               Гарри Поттер и фи...|J.K. Rowling|        6807364|
|         

**9.**

In [188]:
(df
     .groupBy("RatingDistTotal").count()
     .orderBy("count", ascending=False)
     .show(5)
)

+---------------+------+
|RatingDistTotal| count|
+---------------+------+
|              0|451783|
|              1|185753|
|              2|114184|
|              3| 80470|
|              4| 61304|
+---------------+------+
only showing top 5 rows

